In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/DataFusionContest/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Все, что выше - удалить

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import io
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import pickle

SLICE = 1000000

In [ ]:
# Чтение датасета
df = pd.read_csv('clean.csv')
df['item_name'] = df['item_name'].astype(str)
df['item_name'] = df['item_name'].str.lower()

In [ ]:
# Чтение названий колонок результата работы модели
z = pd.read_csv('columns.csv')
dummiesColumnsName = list(z.columns)

In [ ]:
# Загрузка словаря токенайзера
with open('tokenizer_item_name.json') as f:
    data = json.load(f)
    tokenizer_goods = tokenizer_from_json(data)

In [ ]:
# Загрузка модели
model = pickle.load(open('model_v1.sav', 'rb'))

In [ ]:
# Обработка названий товаров по полученному словарю
sequences_goods = tokenizer_goods.texts_to_sequences(df['item_name'])
max_position_len = 10
# Замена неожиданных значений None на 0
for i in range(len(sequences_goods)):
    sequences_goods[i] = [0 if v is None else v for v in sequences_goods[i]]

x1 = pad_sequences(sequences_goods, maxlen=max_position_len)

In [ ]:
del sequences_goods

In [ ]:
zFullPred = pd.DataFrame(columns=['category_id'])
while len(x1) > SLICE:
    x = x1[:SLICE]
    y = model.predict(x)
    yPred = pd.DataFrame(y, columns=dummiesColumnsName)
    zPred = pd.DataFrame(pd.get_dummies(yPred).idxmax(1),
                         columns=['category_id'])
    zPred['category_id'] = zPred['category_id'].astype('uint8')
    zFullPred = zFullPred.append(zPred)
    x1 = x1[SLICE:]
x = x1
y = model.predict(x)
yPred = pd.DataFrame(y, columns=dummiesColumnsName)
zPred = pd.DataFrame(pd.get_dummies(yPred).idxmax(1), columns=['category_id'])
zPred['category_id'] = zPred['category_id'].astype('uint8')
zFullPred = zFullPred.append(zPred)

zFullPred['category_id'] = zFullPred['category_id'].astype('uint8')
zFullPred.to_csv('result.csv', index=False)

In [ ]:
# Удалить все, что после
z_real = pd.DataFrame(df['category_id'].astype('uint8'), columns=['category_id'])

In [ ]:
print(accuracy_score(zFullPred, z_real))

0.9397841444984334
